
# ARBE λ* Studio — **Final Version** (Gradio)
All-in-one App für **Spektral→λ\***, **Matching**, **Paletten**, **Harmonien**, **QA-Reports** und **Exporte**.

> **Hinweis:** Für physikalisch korrekte Illuminanten-Gewichtung (D50/D65/A) und präzise Farbraumkonvertierungen nutzt das Notebook, sofern verfügbar, `colour-science`. Ohne `colour` werden **Equal-Energy**-Gewichtungen und **symbolische** sRGB-Previews verwendet.


In [ ]:

# === Setup (Colab) ===
import sys, os, io, json, math, zipfile, tempfile, shutil, base64, textwrap, pathlib, re
import numpy as np
import pandas as pd

# Optional installs (uncomment in Colab if needed)
try:
    import gradio as gr
except Exception:
    !pip -q install gradio>=4.0.0
    import gradio as gr

# Plotting & reports
try:
    import plotly.express as px
    import plotly.graph_objects as go
except Exception:
    !pip -q install plotly kaleido
    import plotly.express as px
    import plotly.graph_objects as go

# PDF
try:
    from reportlab.lib.pagesizes import A4
    from reportlab.pdfgen import canvas
    from reportlab.lib.units import mm
    from reportlab.lib.utils import ImageReader
except Exception:
    !pip -q install reportlab
    from reportlab.lib.pagesizes import A4
    from reportlab.pdfgen import canvas
    from reportlab.lib.units import mm
    from reportlab.lib.utils import ImageReader

# Optional precise colour science
COLOUR_AVAILABLE = False
try:
    import colour
    COLOUR_AVAILABLE = True
except Exception:
    print("colour-science nicht installiert – Equal-Energy-Fallback aktiv (symbolisch).")

print("Versions:", "numpy", np.__version__, "pandas", pd.__version__)


In [ ]:

# === Utilities ===

def clamp01(x):
    return max(0.0, min(1.0, x))

def srgb_to_rgb1(c):
    # c in 0..255 -> 0..1
    return np.asarray(c, dtype=float)/255.0

def rgb1_to_srgb8(rgb):
    return np.clip(np.round(np.asarray(rgb)*255).astype(int), 0, 255)

# Basic sRGB <-> XYZ <-> Lab (approx., D65; Bradford to D50 only if colour available)
# For accuracy, use colour-science if available
def lab_to_srgb(lab, whitepoint="D50"):
    L,a,b = lab
    if COLOUR_AVAILABLE:
        wp = colour.CCS_ILLUMINANTS["CIE 1931 2 Degree Standard Observer"][whitepoint]
        xyz = colour.Lab_to_XYZ([L,a,b], illuminant=wp)
        # Convert to sRGB (D65)
        rgb = colour.XYZ_to_sRGB(xyz)
        return np.clip(rgb, 0, 1)
    else:
        # simplified approximation via D65 matrix; not colour-accurate
        # Use generic approximate conversion for preview
        # Convert Lab(D50) -> XYZ(D50)
        def f_inv(t):
            d = 6/29
            return t**3 if t> d else 3*d**2*(t-4/29)
        Yn = 1.0; Xn = 0.9642; Zn = 0.8251  # D50 approx
        fy = (L+16)/116; fx = fy + a/500; fz = fy - b/200
        X = Xn * f_inv(fx); Y = Yn * f_inv(fy); Z = Zn * f_inv(fz)
        # Bradford adapt D50->D65 (approx matrix)
        M = np.array([[ 0.9555766, -0.0230393, 0.0631636],
                      [-0.0282895,  1.0099416, 0.0210077],
                      [ 0.0122982, -0.0204830, 1.3299098]])
        X,Y,Z = M @ np.array([X,Y,Z])
        # XYZ(D65) -> linear sRGB
        M2 = np.array([[ 3.2406,-1.5372,-0.4986],
                       [-0.9689, 1.8758, 0.0415],
                       [ 0.0557,-0.2040, 1.0570]])
        rgb_lin = M2 @ np.array([X,Y,Z])
        # gamma companding
        def compand(u):
            return 12.92*u if u<=0.0031308 else 1.055*(u**(1/2.4))-0.055
        rgb = np.array([compand(u) for u in rgb_lin])
        return np.clip(rgb, 0, 1)

def deltaE76(lab1, lab2):
    d = np.array(lab1)-np.array(lab2)
    return float(np.sqrt((d**2).sum()))

def deltaE00(lab1, lab2):
    # Implementation adapted from Sharma et al. 2005 (compact)
    L1,a1,b1 = lab1; L2,a2,b2 = lab2
    avg_L = (L1+L2)/2.0
    C1 = math.hypot(a1,b1); C2 = math.hypot(a2,b2); avg_C = (C1+C2)/2.0
    G = 0.5*(1 - math.sqrt((avg_C**7)/((avg_C**7)+6103515625)))  # 25^7
    a1p = (1+G)*a1; a2p = (1+G)*a2
    C1p = math.hypot(a1p,b1); C2p = math.hypot(a2p,b2); avg_Cp = (C1p+C2p)/2.0
    h1p = math.degrees(math.atan2(b1,a1p)) % 360.0
    h2p = math.degrees(math.atan2(b2,a2p)) % 360.0
    def dhp(h1,h2):
        d = h2-h1
        if d>180: d -= 360
        if d<-180: d += 360
        return d
    dLp = L2-L1
    dCp = C2p-C1p
    dh = dhp(h1p,h2p)
    dHp = 2*math.sqrt(C1p*C2p)*math.sin(math.radians(dh/2))
    avg_hp = (h1p+h2p)/2.0 if abs(h1p-h2p)<=180 else (h1p+h2p+360)/2.0
    T = (1
         - 0.17*math.cos(math.radians(avg_hp-30))
         + 0.24*math.cos(math.radians(2*avg_hp))
         + 0.32*math.cos(math.radians(3*avg_hp+6))
         - 0.20*math.cos(math.radians(4*avg_hp-63)))
    SL = 1 + (0.015*(avg_L-50)**2)/math.sqrt(20+(avg_L-50)**2)
    SC = 1 + 0.045*avg_Cp
    SH = 1 + 0.015*avg_Cp*T
    Rt = -2*math.sqrt((avg_Cp**7)/((avg_Cp**7)+6103515625)) *          math.sin(math.radians(60*math.exp(-(((avg_hp-275)/25)**2))))
    dE = math.sqrt((dLp/SL)**2 + (dCp/SC)**2 + (dHp/SH)**2 + Rt*(dCp/SC)*(dHp/SH))
    return float(dE)

def lch_to_lab(L, C, h_deg):
    a = C * math.cos(math.radians(h_deg))
    b = C * math.sin(math.radians(h_deg))
    return (L, a, b)

def lab_to_lch(L, a, b):
    C = math.hypot(a,b)
    h = (math.degrees(math.atan2(b,a)) + 360) % 360
    return (L, C, h)

def ensure_dataframe(obj):
    if isinstance(obj, pd.DataFrame):
        return obj
    try:
        return pd.read_csv(io.BytesIO(obj))  # uploaded bytes
    except Exception:
        try:
            return pd.read_excel(io.BytesIO(obj))
        except Exception:
            raise ValueError("Ungültiges Tabellenformat (CSV/XLSX erwartet).")


In [ ]:

# === Spectral parsers & λ* ===
def parse_csv_spectra(df):
    # Expect wide or long: either columns 'wavelength' & 'R' + 'sample',
    # or columns: 'sample', '380','390',...
    wl_cols = [c for c in df.columns if re.match(r'^\d{3}$', str(c))]
    if 'wavelength' in df.columns and 'R' in df.columns:
        # long: pivot to wide per sample
        if 'sample' not in df.columns:
            df['sample'] = 'sample_1'
        wide = df.pivot_table(index='sample', columns='wavelength', values='R')
        wide = wide.reset_index().rename_axis(None, axis=1)
        return wide, sorted([int(c) for c in wide.columns if str(c).isdigit()])
    elif len(wl_cols)>0:
        wl = sorted([int(c) for c in wl_cols])
        return df, wl
    else:
        raise ValueError("CSV: Keine spektralen Spalten gefunden.")

def parse_cgats(text):
    rows = []
    current = {}
    wls = []
    for line in io.StringIO(text).read().splitlines():
        s = line.strip()
        if not s or s.startswith('COMMENT'):
            continue
        parts = s.split()
        if len(parts)>=2:
            try:
                wl = int(parts[0]); R = float(parts[1])
                wls.append(wl); rows.append((wl,R))
            except:
                continue
    if not rows:
        raise ValueError("CGATS: Keine Werte erkannt.")
    # Construct single-sample wide table
    wide = pd.DataFrame([dict([('sample','sample_1')]+[(wl,R) for wl,R in rows])])
    return wide, sorted(list(set([r[0] for r in rows])))

def parse_cxf(xml_bytes):
    # Minimal CxF (best-effort): extract (wavelength, R) for first sample
    txt = io.BytesIO(xml_bytes).read().decode('utf-8', errors='ignore')
    wls, Rs = [], []
    for m in re.finditer(r'<Wavelength[^>]*>(\d+)</Wavelength>\s*<Value[^>]*>([0-9\.eE+-]+)</Value>', txt):
        wls.append(int(m.group(1))); Rs.append(float(m.group(2)))
    if not wls:
        # alternative tag names
        for m in re.finditer(r'<WaveLength[^>]*>(\d+)</WaveLength>\s*<Data[^>]*>([0-9\.eE+-]+)</Data>', txt):
            wls.append(int(m.group(1))); Rs.append(float(m.group(2)))
    if not wls:
        raise ValueError("CxF: Keine Werte erkannt.")
    wide = pd.DataFrame([dict([('sample','sample_1')]+list(zip(wls,Rs)))])
    return wide, sorted(list(set(wls)))

def get_illuminant_weights(wavelengths, name='EE'):
    wl = np.array(wavelengths, dtype=float)
    w = np.ones_like(wl)
    if COLOUR_AVAILABLE and name in ['D50','D65','A']:
        spd = colour.SDS_ILLUMINANTS[name]
        w = np.array([spd.value(wi) for wi in wl])
    return w

def lambda_star_for_row(row, wls, illum='EE'):
    R = np.array([float(row[str(w)]) for w in wls], dtype=float)
    W = get_illuminant_weights(wls, illum)
    num = (np.array(wls)*R*W).sum()
    den = (R*W).sum() + 1e-12
    return float(num/den)

def process_spectral_table(wide_df, wls, illum='EE'):
    out = []
    for _,r in wide_df.iterrows():
        lam = lambda_star_for_row(r, wls, illum)
        out.append({'sample': r.get('sample','sample_1'), 'lambda_star': lam})
    return pd.DataFrame(out)


In [ ]:

# === Matching, palette, harmonies ===

def nearest_neighbour(query_df, ref_df, method='DE00', threshold=None):
    # expects columns L,a,b in both
    res = []
    R = ref_df[['L','a','b']].to_numpy(float)
    for _,q in query_df.iterrows():
        qv = np.array([q['L'], q['a'], q['b']], dtype=float)
        # brute-force
        if method=='DE76':
            dists = np.sqrt(((R - qv)**2).sum(axis=1))
        else:
            dists = np.array([deltaE00(qv, rv) for rv in R])
        idx = int(dists.argmin())
        de = float(dists[idx])
        if (threshold is None) or (de <= threshold):
            name = ref_df.iloc[idx].get('name','ref_'+str(idx))
            res.append({'query': q.get('name', f'q_{_}'),
                        'L': q['L'],'a':q['a'],'b':q['b'],
                        'match_name': name,
                        'match_L': ref_df.iloc[idx]['L'],
                        'match_a': ref_df.iloc[idx]['a'],
                        'match_b': ref_df.iloc[idx]['b'],
                        'deltaE': de,
                        'method': method})
    return pd.DataFrame(res)

def kmeans_palette_from_image(img_arr, k=6, max_iter=25):
    # Simple numpy k-means in RGB1 space
    import numpy as np
    data = img_arr.reshape(-1,3).astype(float)/255.0
    # init: random samples
    rng = np.random.default_rng(42)
    cent = data[rng.choice(len(data), size=k, replace=False)]
    for _ in range(max_iter):
        d = ((data[:,None,:]-cent[None,:,:])**2).sum(axis=2)
        lab = d.argmin(axis=1)
        new_cent = np.array([data[lab==i].mean(axis=0) if np.any(lab==i) else cent[i] for i in range(k)])
        if np.allclose(new_cent, cent): break
        cent = new_cent
    return (cent*255).astype(np.uint8)

def lch_harmony(L,C,h, mode='analog', amount=30):
    hs = []
    if mode=='analog':
        hs = [h-amount, h, h+amount]
    elif mode=='komplement':
        hs = [h, (h+180)%360]
    elif mode=='triad':
        hs = [h, (h+120)%360, (h+240)%360]
    elif mode=='tetrad':
        hs = [h, (h+90)%360, (h+180)%360, (h+270)%360]
    return [lch_to_lab(L, C, hh%360) for hh in hs]

def srgb_gamut_flag(lab):
    rgb = lab_to_srgb(lab)
    return bool(np.all((rgb>=0) & (rgb<=1)))


In [ ]:

# === Exports ===

def export_gpl(palette, name="ARBE_palette"):
    # palette: list of dicts {'name':..., 'r':0..255,'g':..,'b':..}
    lines = ["GIMP Palette", "Name: "+name, "Columns: 0", "#"]
    for p in palette:
        lines.append(f"{p['r']} {p['g']} {p['b']} {p.get('name','swatch')}")
    return "\n".join(lines)

def export_aco(palette):
    # Adobe Color Swatch v1 RGB
    from struct import pack
    # build v1 block
    n = len(palette)
    data = pack(">H", n)
    for p in palette:
        r,g,b = p['r'],p['g'],p['b']
        data += pack(">HHHHH", 0, int(r/255*65535), int(g/255*65535), int(b/255*65535), 0)
    # append v2 names
    data += pack(">H", 2)  # version 2
    data += pack(">H", n)
    for p in palette:
        r,g,b = p['r'],p['g'],p['b']
        name = (p.get('name','swatch') + "\x00").encode('utf-16be')
        data += pack(">HHHHH", 0, int(r/255*65535), int(g/255*65535), int(b/255*65535), 0)
        data += pack(">H", len(name)//2)
        data += name
    return data

def export_ase(palette):
    # Minimal ASE RGB (swatches only)
    # palette: [{'name':str, 'r':0..255,'g':..,'b':..}]
    # Build ASE format
    import struct
    def write_block_rgb(name, r,g,b):
        nm = name.encode('utf-16be')
        name_len = len(nm)//2 + 1
        block = struct.pack(">H", 0x0001)  # color entry
        payload = struct.pack(">H", name_len) + nm + b"\x00\x00"
        payload += "RGB ".encode("ascii") + struct.pack(">ffff", r/255, g/255, b/255, 1.0)
        payload += struct.pack(">H", 0)  # color type: global
        block += struct.pack(">I", len(payload)) + payload
        return block
    header = b"ASEF" + struct.pack(">HHI", 1, 0, 0)  # version 1.0, count set later
    blocks = []
    for p in palette:
        blocks.append(write_block_rgb(p.get('name','swatch'), p['r'],p['g'],p['b']))
    count = len(blocks)
    header = b"ASEF" + struct.pack(">HHI", 1, 0, count)
    return header + b"".join(blocks)

def make_qa_pdf(filepath, title, lambda_hist_png=None, table=None):
    c = canvas.Canvas(filepath, pagesize=A4)
    W,H = A4
    c.setFont("Helvetica-Bold", 16); c.drawString(30, H-40, title)
    c.setFont("Helvetica", 10)
    y = H-70
    if lambda_hist_png and os.path.exists(lambda_hist_png):
        img = ImageReader(lambda_hist_png)
        c.drawImage(img, 30, y-220, width=400, height=200, preserveAspectRatio=True, mask='auto')
        y -= 230
    if table is not None:
        txt = c.beginText(30, y)
        txt.textLine("Statistik:")
        for k,v in table.items():
            txt.textLine(f" - {k}: {v}")
        c.drawText(txt)
    c.showPage(); c.save()
    return filepath


In [ ]:

# === Gradio UI ===

REF_TABLE = None  # DataFrame with at least L,a,b[,name]
THEME = "light"

def ui_set_theme(theme):
    global THEME
    THEME = theme
    return f"Theme set to {theme}"

def ui_load_reference(file):
    global REF_TABLE
    if file is None: 
        return "Keine Datei.", None
    content = file.read()
    try:
        df = pd.read_csv(io.BytesIO(content))
    except Exception:
        df = pd.read_excel(io.BytesIO(content))
    # normalize columns
    cols = {c.lower():c for c in df.columns}
    rename = {}
    for want in ['l','a','b','name','h','c','lch']:
        if want in cols and cols[want]!=want:
            rename[cols[want]] = want
    df = df.rename(columns=rename)
    REF_TABLE = df
    return f"Referenz geladen: {len(df)} Zeilen.", df.head(5)

def ui_spectral_lambda(files, illuminant):
    all_rows = []
    bins = []
    for f in files or []:
        name = f.name.lower()
        data = f.read()
        try:
            if name.endswith('.csv'):
                df = pd.read_csv(io.BytesIO(data))
                wide, wls = parse_csv_spectra(df)
            elif name.endswith('.txt'):
                wide, wls = parse_cgats(data.decode('utf-8', errors='ignore'))
            elif name.endswith('.cxf') or name.endswith('.xml'):
                wide, wls = parse_cxf(data)
            elif name.endswith('.zip'):
                # unzip and try csv/txt/xml
                z = zipfile.ZipFile(io.BytesIO(data))
                for zi in z.infolist():
                    if zi.filename.lower().endswith(('.csv','.txt','.cxf','.xml')):
                        with z.open(zi) as fh:
                            subdata = fh.read()
                            # recurse minimalistic: only csv & txt & cxf
                            if zi.filename.lower().endswith('.csv'):
                                df = pd.read_csv(io.BytesIO(subdata))
                                wide, wls = parse_csv_spectra(df)
                            elif zi.filename.lower().endswith('.txt'):
                                wide, wls = parse_cgats(subdata.decode('utf-8','ignore'))
                            else:
                                wide, wls = parse_cxf(subdata)
                            res = process_spectral_table(wide, wls, illum=illuminant)
                            res['source'] = zi.filename
                            all_rows.append(res)
                continue
            else:
                continue
            res = process_spectral_table(wide, wls, illum=illuminant)
            res['source'] = f.name
            all_rows.append(res)
        except Exception as e:
            all_rows.append(pd.DataFrame([{'sample':'ERROR','lambda_star':np.nan,'source':f.name, 'error':str(e)}]))
    if not all_rows:
        return None, None, "Keine gültigen Dateien.", None
    out = pd.concat(all_rows, ignore_index=True)
    # Histogram
    fig = px.histogram(out.dropna(subset=['lambda_star']), x='lambda_star', nbins=60, title='Histogramm λ*')
    return out, fig, f"{len(out)} Einträge verarbeitet.", out.to_csv(index=False)

def ui_match(query_file, method, threshold):
    if REF_TABLE is None:
        return "Bitte zuerst Referenz laden.", None, None
    if query_file is None:
        return "Keine Query-Datei.", None, None
    q = ensure_dataframe(query_file.read())
    # normalize
    q = q.rename(columns={c: c.lower() for c in q.columns})
    for need in ['l','a','b']:
        if need not in q.columns:
            return f"Spalte '{need}' fehlt in Query.", None, None
    q = q.rename(columns={'l':'L','a':'a','b':'b','name':'name'})
    ref = REF_TABLE.rename(columns={c:c if c in ['L','a','b','name'] else c.lower() for c in REF_TABLE.columns})
    if not set(['L','a','b']).issubset(ref.columns):
        return "Referenz hat keine Spalten L,a,b.", None, None
    res = nearest_neighbour(q, ref[['L','a','b','name']], method=('DE76' if method=='ΔE76' else 'DE00'),
                            threshold=(threshold if threshold and threshold>0 else None))
    return f"{len(res)} Matches.", res, res.to_csv(index=False)

def ui_palette(image, k):
    if image is None:
        return "Kein Bild.", None, None, None
    pal = kmeans_palette_from_image(image, k=k)
    rows = []
    for i,(r,g,b) in enumerate(pal):
        name = f"swatch_{i+1}"
        rows.append({'name':name,'r':int(r),'g':int(g),'b':int(b)})
    df = pd.DataFrame(rows)
    gpl = export_gpl(rows, name="ARBE_palette")
    aco = export_aco(rows)
    ase = export_ase(rows)
    return f"{len(rows)} Farben extrahiert.", df, gpl, (aco, ase)

def ui_harmonies(L, C, h, mode):
    labs = lch_harmony(L,C,h, mode=mode)
    rows = []
    for i,lab in enumerate(labs):
        in_gamut = srgb_gamut_flag(lab)
        rgb = rgb1_to_srgb8(lab_to_srgb(lab))
        rows.append({'name':f'h{ i+1 }','L':lab[0],'a':lab[1],'b':lab[2],
                     'in_sRGB': in_gamut, 'r':int(rgb[0]),'g':int(rgb[1]),'b':int(rgb[2])})
    return pd.DataFrame(rows)

with gr.Blocks(title="ARBE λ* Studio — Final") as demo:
    gr.Markdown("## ARBE λ* Studio — Final Version")
    with gr.Row():
        theme = gr.Dropdown(choices=["light","dark"], value="light", label="Theme")
        theme.change(fn=ui_set_theme, inputs=theme, outputs=gr.Textbox(label="Status"))
    with gr.Tab("Referenzen & Setup"):
        ref_file = gr.File(label="Referenz laden (CSV/XLSX: Spalten L,a,b[,name])")
        ref_status = gr.Textbox(label="Status")
        ref_preview = gr.Dataframe(label="Vorschau")
        ref_btn = gr.Button("Referenz laden")
        ref_btn.click(ui_load_reference, inputs=ref_file, outputs=[ref_status, ref_preview])
    with gr.Tab("Spektral → λ*"):
        files = gr.Files(label="Spektraldateien (CSV/CGATS/CxF/ZIP)")
        illum = gr.Dropdown(choices=["D50","D65","A","EE"], value="EE", label="Illuminant für λ*")
        run = gr.Button("λ* berechnen")
        out_table = gr.Dataframe(label="Ergebnisse λ*")
        fig = gr.Plot(label="Histogramm λ*")
        msg = gr.Textbox(label="Meldung")
        out_csv = gr.File(label="Download CSV")
        def _wrap_lambda(files, illum):
            out, fig_, m, csv_txt = ui_spectral_lambda(files, illum)
            csv_path = None
            if csv_txt is not None:
                tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".csv")
                tmp.write(csv_txt.encode("utf-8")); tmp.flush(); tmp.close()
                csv_path = tmp.name
            return out, fig_, m, csv_path
        run.click(_wrap_lambda, inputs=[files, illum], outputs=[out_table, fig, msg, out_csv])
    with gr.Tab("Konvertieren & Suchen"):
        query_file = gr.File(label="Query CSV/XLSX (L,a,b[,name])")
        method = gr.Dropdown(choices=["ΔE00","ΔE76"], value="ΔE00", label="Methode")
        thresh = gr.Slider(0, 20, step=0.1, value=5.0, label="ΔE-Schwelle (optional)")
        btn = gr.Button("Matching starten")
        status = gr.Textbox(label="Status")
        res = gr.Dataframe(label="Matches")
        res_csv = gr.File(label="Download CSV")
        def _wrap_match(q, m, t):
            s, df, csv = ui_match(q, m, t)
            csv_path = None
            if csv is not None:
                tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".csv")
                tmp.write(csv.encode("utf-8")); tmp.flush(); tmp.close()
                csv_path = tmp.name
            return s, df, csv_path
        btn.click(_wrap_match, inputs=[query_file, method, thresh], outputs=[status, res, res_csv])
    with gr.Tab("Bild → Palette & Exporte"):
        img = gr.Image(type="numpy", label="Bild laden")
        k = gr.Slider(2, 12, step=1, value=6, label="Anzahl Farben")
        runp = gr.Button("Palette extrahieren")
        info = gr.Textbox(label="Status")
        pal = gr.Dataframe(label="Palette (RGB 0..255)")
        gpl = gr.Textbox(label="GPL (Text)")
        aco_file = gr.File(label="ACO")
        ase_file = gr.File(label="ASE")
        def _wrap_pal(image, k):
            s, df, gpl_txt, (aco_bytes, ase_bytes) = ui_palette(image, k)
            aco_path = tempfile.NamedTemporaryFile(delete=False, suffix=".aco"); open(aco_path.name,'wb').write(aco_bytes)
            ase_path = tempfile.NamedTemporaryFile(delete=False, suffix=".ase"); open(ase_path.name,'wb').write(ase_bytes)
            return s, df, gpl_txt, aco_path.name, ase_path.name
        runp.click(_wrap_pal, inputs=[img, k], outputs=[info, pal, gpl, aco_file, ase_file])
    with gr.Tab("Harmonien (LCh)"):
        L = gr.Slider(0, 100, value=60, label="L*")
        C = gr.Slider(0, 150, value=40, label="C")
        h = gr.Slider(0, 360, value=30, label="h°")
        mode = gr.Dropdown(choices=["analog","komplement","triad","tetrad"], value="analog", label="Schema")
        go_btn = gr.Button("Erzeugen")
        res_tab = gr.Dataframe(label="Ergebnis (mit sRGB-Gamut-Flag)")
        go_btn.click(lambda L,C,h,mode: ui_harmonies(L,C,h,mode), inputs=[L,C,h,mode], outputs=res_tab)

# Auto-launch in Colab
try:
    import google.colab  # type: ignore
    demo.launch(share=False)
except Exception:
    print("Starte App lokal/Notebook: demo.launch() ausführen.")
